# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
!pip install -U -q sacremoses

# Homophone List

This needs to be expanded.

In [23]:
# Chat GPT Prompt: Give me a list of lists in python of 100 sets of homophones
homophones_list = [
    ["ate", "eight"],
    ["bare", "bear"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["cell", "sell"],
    ["cite", "site", "sight"],
    ["complement", "compliment"],
    ["desert", "dessert"],
    ["die", "dye"],
    ["flour", "flower"],
    ["hear", "here"],
    ["hour", "our"],
    ["knight", "night"],
    ["know", "no"],
    ["mail", "male"],
    ["meat", "meet"],
    ["morning", "mourning"],
    ["one", "won"],
    ["pair", "pear"],
    ["peace", "piece"],
    ["principal", "principle"],
    ["rain", "reign", "rein"],
    ["right", "write"],
    ["sea", "see"],
    ["serial", "cereal"],
    ["sole", "soul"],
    ["stationary", "stationery"],
    ["tail", "tale"],
    ["threw", "through"],
    ["to", "too", "two"],
    ["weather", "whether"],
    ["week", "weak"],
    ["wear", "where"],
    ["which", "witch"],
    ["your", "you're"],
    ["allowed", "aloud"],
    ["board", "bored"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["compliment", "complement"],
    ["desert", "dessert"],
    ["dual", "duel"],
    ["fair", "fare"],
    ["genre", "jinja"],
    ["hare", "hair"],
    ["here", "hear"],
    ["hoard", "horde"],
    ["loan", "lone"],
    ["pail", "pale"],
    ["peak", "peek", "pique"],
    ["profit", "prophet"],
    ["role", "roll"],
    ["root", "route"],
    ["sail", "sale"],
    ["scene", "seen"],
    ["serial", "cereal"],
    ["so", "sow"],
    ["stare", "stair"],
    ["steal", "steel"],
    ["their", "there", "they're"],
    ["throne", "thrown"],
    ["vain", "vein", "vane"],
    ["weak", "week"],
    ["wood", "would"],
    ["yew", "you"],
    ["bridal", "bridle"],
    ["cereal", "serial"],
    ["chord", "cord"],
    ["compliment", "complement"],
    ["dew", "due"],
    ["foul", "fowl"],
    ["grate", "great"],
    ["groan", "grown"],
    ["heal", "heel"],
    ["him", "hymn"],
    ["lay", "lie"],
    ["main", "mane"],
    ["marry", "merry"],
    ["mite", "might"],
    ["moose", "mousse"],
    ["mourn", "morn"],
    ["peace", "piece"],
    ["plum", "plumb"],
    ["pour", "pore"],
    ["rap", "wrap"],
    ["scene", "seen"],
    ["scent", "cent", "sent"],
    ["serial", "cereal"],
    ["shear", "sheer"],
    ["soar", "sore"],
    ["sow", "sew"],
    ["stake", "steak"],
    ["tide", "tied"],
    ["toe", "tow"],
    ["there", "their", "they're"],
    ["waist", "waste"],
    ["week", "weak"],
    ["write", "right", "rite"],
    ["won", "one"],
    ["your", "you're"],
    ["dear", "deer"],
    ["affect", "effect"],
    ["aisle", "isle"],
    ["all", "awl"],
    ["allowed", "aloud"],
    ["altar", "alter"],
    ["arc", "ark"],
    ["ascent", "assent"],
    ["son", "sun"]
]


# Load Model

Bert has been working decently well for me.

In [5]:
from transformers import pipeline

# Load the model and tokenizer
fill_mask = pipeline('fill-mask', model='bert-base-uncased')

/Users/Austin/opt/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Test case 

In [6]:
# Create input string
input_string = "Michigan one the football game against Ohio State!"

# Select homophone
target_homophone = "one"

# Get homophone options
homophone_options = [homophone for homophone in homophones_list if target_homophone in homophone]

# Replace homophone with mask token
input_string = input_string.replace(target_homophone, fill_mask.tokenizer.mask_token)

# Get results
results = fill_mask(input_string)

# Token string dict
token_string_dict = {}

# Get top results
for result in results:
   token_string_dict[result["token_str"]] = result["score"]

# Sort results
sorted_results = sorted(token_string_dict.items(), key=lambda x: x[1], reverse=True)

# Find top homophone in results
homophone_results = [result for result in sorted_results if result[0] in homophone_options[0]]

if homophone_results[0][0] == target_homophone:
    print("Correct!")
else:
    print("Incorrect!")
    print("Top result: ", homophone_results[0][0])
    print("Top result score: ", homophone_results[0][1])

Incorrect!
Top result:  won
Top result score:  0.704522430896759


## Building a function

In [7]:
# Function that can check a sentence that only contains one homophone
def one_homophone_checker(input_string, homophones_list=homophones_list, score_threshold=0):
    # Lowering the input string
    input_string = input_string.lower()

    # Flatten homophones list
    all_homophones = [
        word for homophone_set in homophones_list for word in homophone_set
    ]

    # Find homophones in input string
    target_homophone = list(set(input_string.split(" ")).intersection(set(all_homophones)))
    print(target_homophone)

    # If there are no homophones in the sentence, return the NA dataframe
    if len(target_homophone) < 1:
        output_df = pd.DataFrame(
            {
                "sentence": input_string,
                "has_homophone": False,
                "is_error": None,
                "error_idx": None,
                "error": None,
                "correct_word": None,
                "correct_sentence": None,
            }, index=[0]
        )
        return output_df

    else:
        target_homophone = target_homophone[0]

    # Get all homophone options from homophones list
    homophone_options = [homophone for homophone in homophones_list if target_homophone in homophone]

    # Replace homophone with mask token
    # CANNOT HANDLE MULTIPLE HOMOPHONES IN ONE SENTENCE
    masked_string = ' '.join([word if word != target_homophone else fill_mask.tokenizer.mask_token for word in input_string.split(" ")])
    print(masked_string)
    
    # Get results
    results = fill_mask(masked_string, top_k=100)

    # Token string dict
    token_string_dict = {}

    # Get top results and their score
    for result in results:
        try:
            token_string_dict[result["token_str"]] = result["score"]
        except TypeError:
            print(result)

    # Sort results
    sorted_results = sorted(token_string_dict.items(), key=lambda x: x[1], reverse=True)

    # Find top homophone in results
    homophone_results = [result for result in sorted_results if result[0] in homophone_options[0]]
    print(homophone_results)
    
    # If the top result is the target homophone, return the original sentence as it is correct
    if homophone_results[0][0] == target_homophone:
        final_sentence = input_string
        is_error = False
        correct_word = None
        error_idx = None
        error = None

    else:
        # If the top result is not the target homophone, check how many options it found
        # If multiple options:
        if len(homophone_results) > 1:

            # Check the difference between the top two results
            score_diff = homophone_results[0][1] - homophone_results[1][1]

            # If the difference is greater than the threshold, return the top result
            if score_diff > score_threshold:
                error_idx = input_string.split(" ").index(target_homophone)
                error = target_homophone
                final_sentence = input_string.replace(target_homophone, homophone_results[0][0])
                correct_word = homophone_results[0][0]
                is_error = True
        
        # If the difference is less than the threshold, return the original sentence
        else:
            is_error = True
            error_idx = input_string.split(" ").index(target_homophone)
            final_sentence = input_string.replace(target_homophone, homophone_results[0][0])
            correct_word = homophone_results[0][0]
            error = target_homophone

    # Create output dataframe that matches our test data
    output_df = pd.DataFrame(
            {
                "sentence": input_string,
                "has_homophone": True,
                "is_error": is_error,
                "error_idx": error_idx,
                "error": error,
                "correct_word": correct_word,
                "correct_sentence": final_sentence,
            }, index=[0]
        )
    return output_df

## Testing

In [8]:
input_string = "At lunch today, I had way to much food!"

one_homophone_checker(input_string)

['to']
at lunch today, i had way [MASK] much food!
[('too', 0.9997673630714417), ('to', 4.6690133785887156e-06)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,"at lunch today, i had way to much food!",True,True,6,to,too,"at lunch tooday, i had way too much food!"


In [9]:
input_string = "Michigan one the game against Ohio State!"

one_homophone_checker(input_string)

['one']
michigan [MASK] the game against ohio state!
[('won', 0.7407358884811401)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,michigan one the game against ohio state!,True,True,1,one,won,michigan won the game against ohio state!


# NEXT STEPS: MAKE SENTENCES LIKE THIS WORK WITH MULTIPLE ERRORS

In [10]:
input_string = "Have you scene my new waist of money?"

one_homophone_checker(input_string)

['you', 'waist', 'scene']
have [MASK] scene my new waist of money?
[('you', 0.3468347489833832)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,have you scene my new waist of money?,True,False,None,None,None,have you scene my new waist of money?


In [60]:
from tqdm import tqdm
# Function that can check a sentence that has multiple homophones
def homophone_checker(input_string, homophones_list=homophones_list, score_threshold=0):
    # Lowering the input string
    input_string = input_string.lower()
    input_string_list = input_string.split(" ")
    # Flatten homophones list
    all_homophones = [
        word for homophone_set in homophones_list for word in homophone_set
    ]

    # Find homophones in input string
    target_homophones = [(word, i) for i, word in enumerate(input_string_list) if list(set([word]).intersection(set(all_homophones)))]
    # print(target_homophones)

    # If there are no homophones in the sentence, return the NA dataframe
    if len(target_homophones) < 1:
        output_df = pd.DataFrame(
            {
                "sentence": input_string,
                "has_homophone": False,
                "is_error": None,
                "error_idx": None,
                "error": None,
                "correct_word": None,
                "correct_sentence": None,
            }, index=[0]
        )
        return output_df

    else:
        final_sentence = []
        is_error = []
        correct_word = []
        error_idx = []
        error = []

        for target_homophone_tuple in tqdm(target_homophones):
            target_homophone = target_homophone_tuple[0]
            target_homophone_idx = target_homophone_tuple[1]

            input_string_list = input_string.split(" ")
            
            # Get all homophone options from homophones list
            homophone_options = [homophone for homophone in homophones_list if target_homophone in homophone]

            # Replace homophone with mask token
            input_string_list[target_homophone_idx] = fill_mask.tokenizer.mask_token
            masked_string = ' '.join(input_string_list)
            # print(masked_string)
            
            # Get results
            results = fill_mask(masked_string, top_k=100)

            # Token string dict
            token_string_dict = {}

            # Get top results and their score
            for result in results:
                try:
                    token_string_dict[result["token_str"]] = result["score"]
                except TypeError:
                    # print(result)
                    pass

            # Sort results
            sorted_results = sorted(token_string_dict.items(), key=lambda x: x[1], reverse=True)

            # Find top homophone in results
            homophone_results = [result for result in sorted_results if result[0] in homophone_options[0]]
            print(homophone_results)
            
            # If the top result is the target homophone, return the original sentence as it is correct
            try:
                top_result = homophone_results[0][0]
            except IndexError:
                final_sentence.append(input_string)
                is_error.append(False)
                correct_word.append(None)
                error_idx.append(None)
                error.append(None)
                continue

            if top_result == target_homophone:
                final_sentence.append(input_string)
                is_error.append(False)
                correct_word.append(None)
                error_idx.append(None)
                error.append(None)

            else:
                # If the top result is not the target homophone, check how many options it found
                # If multiple options:
                if len(homophone_results) > 1:

                    # Check the difference between the top two results
                    score_diff = homophone_results[0][1] - homophone_results[1][1]

                    # If the difference is greater than the threshold, return the top result
                    if score_diff > score_threshold:
                        error_idx.append(input_string.split(" ").index(target_homophone))
                        error.append(target_homophone)
                        final_sentence.append(input_string.replace(target_homophone, homophone_results[0][0]))
                        correct_word.append(homophone_results[0][0])
                        is_error.append(True)
                
                # If the difference is less than the threshold, return the original sentence
                else:
                    is_error.append(True)
                    error_idx.append(input_string.split(" ").index(target_homophone))
                    final_sentence.append(input_string.replace(target_homophone, homophone_results[0][0]))
                    correct_word.append(homophone_results[0][0])
                    error.append(target_homophone)

        # Create output dataframe that matches our test data
        output_df = pd.DataFrame(
                {
                    "sentence": input_string,
                    "has_homophone": True,
                    "is_error": is_error,
                    "error_idx": error_idx,
                    "error": error,
                    "correct_word": correct_word,
                    "correct_sentence": final_sentence,
                }, index=range(len(target_homophones))
            )
        return output_df

In [48]:
homophone_checker("I drove to the restaurant to get some food but I eight way too much.")

[('to', 2), ('to', 5), ('eight', 11), ('too', 13)]
i drove [MASK] the restaurant to get some food but i eight way too much.
[('to', 0.8983819484710693), ('too', 6.669076356047299e-06)]
i drove to the restaurant [MASK] get some food but i eight way too much.
[('to', 0.999870777130127), ('too', 8.88586100700195e-07)]
i drove to the restaurant to get some food but i [MASK] way too much.
[('ate', 0.6147705316543579)]
i drove to the restaurant to get some food but i eight way [MASK] much.
[('too', 0.9958474040031433), ('to', 0.00022882445773575455)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,i drove to the restaurant to get some food but...,True,False,NaN,None,None,i drove to the restaurant to get some food but...
1,i drove to the restaurant to get some food but...,True,False,NaN,None,None,i drove to the restaurant to get some food but...
2,i drove to the restaurant to get some food but...,True,True,11.0,eight,ate,i drove to the restaurant to get some food but...
3,i drove to the restaurant to get some food but...,True,False,NaN,None,None,i drove to the restaurant to get some food but...


In [55]:
homophone_checker("We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed too suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.")

100%|██████████| 34/34 [00:31<00:00,  1.09it/s]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
1,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
2,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
3,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
4,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
5,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
6,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
7,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
8,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."
9,"we hold these truths to be self-evident, that ...",True,False,NaN,None,None,"we hold these truths to be self-evident, that ..."


Correct but slow. I added the wrong "too" and the affect one is close.

In [56]:
homophone_checker("Natural language processing (NLP) lies at the heart of modern information systems. The rate of advancement in the field is measurably exponential; in the last 30 years it has transformed how humans acquire knowledge and interact with computers, multiple times. This course traces these advancements through the lens of the statistical machine learning methods that have enabled them. We explore how language understanding is framed as a tractable inference problem through language modeling, and the model architectures and algorithms that have proven most successful in real world applications. This course assumes a basic understanding of linear algebra, probability theory, and first order optimization methods.")

100%|██████████| 2/2 [00:01<00:00,  1.48it/s]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,natural language processing (nlp) lies at the ...,True,False,None,None,None,natural language processing (nlp) lies at the ...
1,natural language processing (nlp) lies at the ...,True,False,None,None,None,natural language processing (nlp) lies at the ...


In [57]:
# Changed "through" to "threw"
homophone_checker("Natural language processing (NLP) lies at the heart of modern information systems. The rate of advancement in the field is measurably exponential; in the last 30 years it has transformed how humans acquire knowledge and interact with computers, multiple times. This course traces these advancements through the lens of the statistical machine learning methods that have enabled them. We explore how language understanding is framed as a tractable inference problem threw language modeling, and the model architectures and algorithms that have proven most successful in real world applications. This course assumes a basic understanding of linear algebra, probability theory, and first order optimization methods.")

100%|██████████| 2/2 [00:01<00:00,  1.01it/s]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,natural language processing (nlp) lies at the ...,True,False,NaN,None,None,natural language processing (nlp) lies at the ...
1,natural language processing (nlp) lies at the ...,True,True,70.0,threw,through,natural language processing (nlp) lies at the ...


In [61]:
# Change "due" to "dew" in last sentence
homophone_checker("Late Policy: All submissions are due 11:59pm EST on the dates listed. You are free to submit work up to 7 days after the due date, subject to the following: All submissions will be assigned a score equal to the higher of (a) the grade of the on-time submission, and (b) the grade of the late submission with a 20% deduction. As an example, if you were to receive an 79% on a lab, and then submit changes with corrections within the 7 day window resulting in a revised nominal score of 100%, you will receive a grade of 80% for that lab. Importantly, this late policy does not apply to the final project; the dew date for final project submissions will be strictly enforced.")

  7%|▋         | 1/14 [00:00<00:10,  1.28it/s]

[('all', 0.9307690262794495)]


 14%|█▍        | 2/14 [00:01<00:06,  1.73it/s]

[('due', 0.058565735816955566)]


 21%|██▏       | 3/14 [00:01<00:05,  1.96it/s]

...

 93%|█████████▎| 13/14 [00:05<00:00,  2.31it/s]

[('to', 0.9436678290367126), ('too', 3.871240551234223e-06)]


100%|██████████| 14/14 [00:06<00:00,  2.20it/s]

[('due', 0.9452741146087646)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
1,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
2,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
3,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
4,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
5,late policy: all submissions are due 11:59pm e...,True,True,5.0,due,dew,late policy: all submissions are dew 11:59pm e...
6,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
7,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
8,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...
9,late policy: all submissions are due 11:59pm e...,True,False,NaN,None,None,late policy: all submissions are due 11:59pm e...


Found my change but also wrongly changed one earlier; I think we will need to add a threshold.

In [63]:
homophone_checker("Considering Meta’s strategic shift too the metaverse, encompassing Facebook’s social media and related services, the current vision statement accurately represents the business and its operations. On the other hand, the mission statement remains focused on building connections. These factors indicate that the corporate mission and vision statements guide Facebook’s (Meta’s) operations management to support business growth based on the metaverse.")

 50%|█████     | 1/2 [00:00<00:00,  1.27it/s]

[('to', 0.5008355975151062)]


100%|██████████| 2/2 [00:01<00:00,  1.89it/s]

[('to', 0.4922170639038086)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,considering meta’s strategic shift too the met...,True,True,4.0,too,to,considering meta’s strategic shift to the meta...
1,considering meta’s strategic shift too the met...,True,False,NaN,None,None,considering meta’s strategic shift too the met...


In [64]:
# Changed "sleigh" to "slay"
homophone_checker("Rudolph the Red-Nosed Reindeer is a fictional reindeer created by Robert L. May. Rudolph is usually depicted as the ninth and youngest of Santa Claus's reindeer, using his luminous red nose to lead the reindeer team and guide Santa's slay on Christmas Eve.")

100%|██████████| 1/1 [00:00<00:00,  1.97it/s]

[('to', 0.9995779395103455)]


,sentence,has_homophone,is_error,error_idx,error,correct_word,correct_sentence
0,rudolph the red-nosed reindeer is a fictional ...,True,False,None,None,None,rudolph the red-nosed reindeer is a fictional ...


Undetected; our homophone list is too short.